# GUC Clustering Project 

Ahmad Ashraf Ahmad
46-3060
T-6

**Objective:** 
The objective of this project teach students how to apply clustering to real data sets

The projects aims to teach student: 
* Which clustering approach to use
* Compare between Kmeans, Hierarchal, DBScan, and Gaussian Mixtures  
* How to tune the parameters of each data approach
* What is the effect of different distance functions (optional) 
* How to evaluate clustering approachs 
* How to display the output
* What is the effect of normalizing the data 

Students in this project will use ready-made functions from Sklearn, plotnine, numpy and pandas 
 



In [ ]:
# if plotnine is not installed in Jupter then use the following command to install it 
!pip install plotnine

Running this project require the following imports 

In [ ]:
pip install plotnine

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.preprocessing as prep
from sklearn.datasets import make_blobs
from plotnine import *   
# StandardScaler is a function to normalize the data 
# You may also check MinMaxScaler and MaxAbsScaler 
#from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

from sklearn.cluster import DBSCAN


from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_score

%matplotlib inline
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn import preprocessing


In [ ]:
# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            plt.scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)

In [ ]:
# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster_h(X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],cmap = 'plasma',alpha = alpha,s=s)

## Multi Blob Data Set 
* The Data Set generated below has 6 cluster with varying number of users and varing densities
* Cluster the data set below using 



In [ ]:
plt.rcParams['figure.figsize'] = [8,8]
sns.set_style("whitegrid")
sns.set_context("talk")

n_bins = 6  
centers = [(-3, -3), (0, 0), (5,2.5),(-1, 4), (4, 6), (9,7)]
Multi_blob_Data, y = make_blobs(n_samples=[100,150, 300, 400,300, 200], n_features=2, cluster_std=[1.3,0.6, 1.2, 1.7,0.9,1.7],
                  centers=centers, shuffle=False, random_state=42)
display_cluster(Multi_blob_Data)

### Kmeans 
* Use Kmeans with different values of K to cluster the above data 
* Display the outcome of each value of K 
* Plot distortion function versus K and choose the approriate value of k 
* Plot the silhouette_score versus K and use it to choose the best K 
* Store the silhouette_score for the best K for later comparison with other clustering techniques. 

#### Kmeans with different K values
 

In [ ]:
df=pd.DataFrame(Multi_blob_Data)
plt.figure(figsize=(50, 25))
for i in range(1,15):
    kmeans = KMeans(n_clusters=i,random_state=42)
    kmeans.fit(Multi_blob_Data)
    plt.subplot(3, 5, i)
    plt.scatter(x=df[0],y=df[1],c=kmeans.labels_)

##### Elbow Method

In [ ]:
inertias = []

for i in range(1,15):
    kmeans = KMeans(n_clusters=i,random_state=42)
    kmeans.fit(Multi_blob_Data)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,15), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

##### K=5

##### Silhouette Method

In [ ]:
scores = []
for i in range(2,15):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit_predict(Multi_blob_Data)
    scores.append(silhouette_score(Multi_blob_Data, kmeans.labels_, metric='euclidean'))

plt.plot(range(2,15), scores, marker='o')
plt.title('Silhouette method')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
plt.show()

##### K=6


Documentation: 
metricstr or callable, default=’euclidean’
The metric to use when calculating distance between instances in a feature array. If metric is a string, it must be one of the options allowed by metrics.pairwise.pairwise_distances. If X is the distance array itself, use metric="precomputed"

### Hierarchal Clustering
* Use AgglomerativeClustering function to  to cluster the above data 
* In the  AgglomerativeClustering change the following parameters 
    * Affinity (use euclidean, manhattan and cosine)
    * Linkage( use average and single )
    * Distance_threshold (try different)
* For each of these trials plot the Dendograph , calculate the silhouette_score and display the resulting clusters  
* Find the set of paramters that would find result in the best silhouette_score and store this score for later comparison with other clustering techniques. 
* Record your observation 

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
df=pd.DataFrame(Multi_blob_Data)
plt.figure(figsize=(100, 100))
scores=[]
clusters=[]
parameters=[]
linkage_array=["average","single"]
affinity_array=["euclidean","manhattan","cosine"]
z=1
for k in range(1,3):
    link=linkage_array[k-1]
    for l in range (1,4):
        a=affinity_array[l-1]
        if link=="average" and a=="cosine":
            start=0.3
            stop=0.8
            step=0.1
        elif link=="single" and a=="cosine":
            start=0.05
            stop=0.1
            step=0.01            
        elif link=="single" and (a=="manhattan" or a=="euclidean"):
            start=0.05
            stop=0.1
            step=0.01
        else:
            start=1
            stop=6
            step=1
        for m in np.arange (start,stop,step):
            d=m
            clt = AgglomerativeClustering(linkage=link, affinity=a,distance_threshold=d,n_clusters=None)
            model = clt.fit(Multi_blob_Data)
            if (len(np.unique(model.labels_)))>1:
                plt.subplot(11, 5,z)
                plot_dendrogram(model, truncate_mode="level", p=8)
                plt.title(str(link)+" , "+str(a)+" , "+str(d))
                plt.subplot(11, 5,z+1)
                plt.scatter(x=df[0],y=df[1],c=model.labels_)
                plt.title(str(link)+" , "+str(a)+" , "+str(d))
                scores.append(silhouette_score(df[[0,1]],model.labels_ ))
                clusters.append(len(np.unique(model.labels_)))
                parameters.append(str(link)+" , "+str(a)+" , "+str(d))
                z=z+2
plt.subplot(11,5,z)
plt.plot(clusters, scores, marker='o')
plt.title('Silhouette method')
plt.xlabel('Number of clusters')
plt.ylabel('Score')
# print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
#       " and parameters "+str(parameters[scores.index(max(scores))]))


### DBScan
* Use DBScan function to  to cluster the above data 
* In the  DBscan change the following parameters 
    * EPS (from 0.1 to 3)
    * Min_samples (from 5 to 25)
* Plot the silhouette_score versus the variation in the EPS and the min_samples
* Plot the resulting Clusters in this case 
* Find the set of paramters that would find result in the best silhouette_score and store this score for later comparison with other clustering techniques. 
* Record your observations and comments 

Silhouette Method: This technique measures the separability between clusters. First, an average distance is found between each point and all other points in a cluster. Then it measures the distance between each point and each point in other clusters. We subtract the two average measures and divide by whichever average is larger.
We ultimately want a high (ie. closest to 1) score which would indicate that there is a small intra-cluster average distance (tight clusters) and a large inter-cluster average distance (clusters well separated).

In [ ]:
scores=[]
clusters=[]
parameters=[]
EPSS=[]
Min_Sampless=[]
df=pd.DataFrame(Multi_blob_Data)
z=1
plt.figure(figsize=(100, 100))
for Min_Samples in range (5,26):
    for EPS in np.arange (0.1,4,0.3):
        model = (DBSCAN(eps=EPS, min_samples=Min_Samples)).fit(Multi_blob_Data)
        if (len(np.unique(model.labels_)))>1:
            plt.subplot(17, 10,z)
            plt.scatter(x=df[0],y=df[1],c=model.labels_)
            plt.title("EPS="+str(EPS)+" Min_Samples= "+str(Min_Samples))
            scores.append(silhouette_score(df,model.labels_ ))
            clusters.append(len(np.unique(model.labels_)))
            EPSS.append(EPS)
            Min_Sampless.append(Min_Samples)
            parameters.append(str(EPS)+" , "+str(Min_Samples))
            z=z+1
plt.subplot(17,10,z)
plt.plot(EPSS, scores, marker='o')
plt.title("EPS Variation with Silhouette Score ")
plt.xlabel('EPS')
plt.ylabel('Score')
plt.subplot(17,10,z+1)
plt.plot(Min_Sampless, scores, marker='o')
plt.title("Min_points Variation with Silhouette Score ")
plt.xlabel('Min_points')
plt.ylabel('Score')
print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
      " and parameters "+str(parameters[scores.index(max(scores))]))

In [ ]:
maximum_ss=max(scores)
maximum_index=scores.index(maximum_ss)
best_eps=EPSS[maximum_index]
best_MinPoints=Min_Sampless[maximum_index]
print("Best EPS=",best_eps,"and best MinPoints=",best_MinPoints,"for the best silhoutte score of",maximum_ss)

### Gaussian Mixture
* Use GaussianMixture function to cluster the above data 
* In GMM change the covariance_type and check the difference in the resulting proabability fit 
* Use a 2D contour plot to plot the resulting distribution (the components of the GMM) as well as the total Gaussian mixture 

In [ ]:
from scipy.stats import multivariate_normal
covariance = ['spherical', 'diag', 'tied', 'full']
p=0
plt.figure(figsize=(50, 25))
for x in covariance:
    # Initialize the GMM model.
    GMM_Blob = GaussianMixture(n_components=2, covariance_type=x, random_state=0).fit(Multi_blob_Data)
    labels = GMM_Blob.predict(Multi_blob_Data)
    
    # Define a grid to plot the contour plot
    x_min, x_max = Multi_blob_Data[:, 0].min() - 1, Multi_blob_Data[:, 0].max() + 1
    y_min, y_max = Multi_blob_Data[:, 1].min() - 1, Multi_blob_Data[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    Z = np.zeros((xx.shape[0], xx.shape[1]))
    p=p+1
    for i in range(GMM_Blob.n_components):
        density = multivariate_normal(mean=GMM_Blob.means_[i], cov=GMM_Blob.covariances_[i]).pdf(np.dstack((xx, yy)))
        Z += density * GMM_Blob.weights_[i]
    
    # Plot the resulting contour plot
    plt.subplot(2,2,p)
    plt.contour(xx, yy, Z, levels=10, linewidths=1, colors='black')
    plt.contourf(xx, yy, Z, levels=10, cmap='tab20b')
    plt.scatter(Multi_blob_Data[:, 0], Multi_blob_Data[:, 1], c=GMM_Blob.predict(Multi_blob_Data), cmap='tab20b')
    plt.title('Gaussian Mixture Clustering using %s' % x)

## iris data set 
The iris data set is test data set that is part of the Sklearn module 
which contains 150 records each with 4 features. All the features are represented by real numbers 

The data represents three classes 


* Repeat all the above clustering approaches and steps on the above data 
* Normalize the data then repeat all the above steps 
* Compare between the different clustering approaches 

## Before Normalization

In [ ]:
from sklearn.datasets import load_iris
iris_data = load_iris()
iris_data.target[[10, 25, 50]]
#array([0, 0, 1])
# list(iris_data.target_names)
# ['setosa', 'versicolor', 'virginica']

# df = pd.DataFrame(iris_data, columns = ['Sepal Length','Sepal Width','Petal Length','Petal Width']) 
# df
df = pd.DataFrame(data= np.c_[iris_data['data']],
                     columns= iris_data['feature_names'] )
df

## Kmeans

In [ ]:
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(50, 25))
            plt.title(Features_string)
            
        else:
            continue
        inertias = []
        scores = []
        clusters_string=[]
        clusters=[]
        for num_clusters in range(2,18):
            kmeans = KMeans(n_clusters=num_clusters,random_state=42)
            kmeans.fit(df[[df.columns[i],df.columns[j]]])
            inertias.append(kmeans.inertia_)
            scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]], kmeans.labels_, metric='euclidean'))
            clusters_string=str(df.columns[i])+" and "+str(df.columns[j])+" for "+str(num_clusters)+" Clusters "
            clusters.append(len(np.unique(kmeans.labels_)))
            plt.subplot(5, 4, num_clusters-1)
            plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=kmeans.labels_)
            plt.title(clusters_string)
        plt.subplot(5,4,num_clusters)
        plt.plot(range(2,num_clusters+1), inertias, marker='o')
        plt.title('Elbow method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Inertia')
        plt.subplot(5,4,num_clusters+1)
        plt.plot(range(2,num_clusters+1), scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print("Based on silhouette scores, the best number of clusters for "+ Features_string +" is "+str(clusters[scores.index(max(scores))]))

### Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
scores=[]
clusters=[]
parameters=[]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        linkage_array=["average","single"]
        affinity_array=["euclidean","manhattan","cosine"]
        z=1
        for k in range(1,3):
            link=linkage_array[k-1]
            for l in range (1,4):
                a=affinity_array[l-1]
                if link=="average" and a=="cosine":
                    start=0.3
                    stop=0.8
                    step=0.1
                elif link=="single" and a=="cosine":
                    start=0.05
                    stop=0.1
                    step=0.01            
                elif link=="single" and (a=="manhattan" or a=="euclidean"):
                    start=0.05
                    stop=0.1
                    step=0.01
                else:
                    start=1
                    stop=6
                    step=1
                for m in np.arange (start,stop,step):
                    d=m
                    clt = AgglomerativeClustering(linkage=link, affinity=a,distance_threshold=d,n_clusters=None)
                    model = clt.fit(df[[df.columns[i],df.columns[j]]])
                    if (len(np.unique(model.labels_)))>1:
                        plt.subplot(13, 5,z)
                        plot_dendrogram(model, truncate_mode="level", p=8)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        plt.subplot(13, 5,z+1)
                        plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=model.labels_)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                        clusters.append(len(np.unique(model.labels_)))
                        parameters.append(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        z=z+2
        plt.subplot(13,5,z)
        plt.plot(clusters, scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))


### DBSCAN

In [ ]:
scores=[]
clusters=[]
parameters=[]
EPSS=[]
Min_Sampless=[]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        EPSS=[]
        Min_Sampless=[]
        z=1
        for Min_Samples in range (5,26):
            for EPS in np.arange (0.1,4,0.3):
                model = (DBSCAN(eps=EPS, min_samples=Min_Samples)).fit(df[[df.columns[i],df.columns[j]]])
                if (len(np.unique(model.labels_)))>1:
                    plt.subplot(21, 10,z)
                    plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=model.labels_)
                    plt.title(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                    clusters.append(len(np.unique(model.labels_)))
                    EPSS.append(EPS)
                    Min_Sampless.append(Min_Samples)
                    parameters.append(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    z=z+1
        plt.subplot(21,10,z)
        plt.plot(EPSS, scores, marker='o')
        plt.title("EPS Variation with Silhouette Score ")
        plt.xlabel('EPS')
        plt.ylabel('Score')
        plt.subplot(21,10,z+1)
        plt.plot(Min_Sampless, scores, marker='o')
        plt.title("Min_points Variation with Silhouette Score ")
        plt.xlabel('Min_points')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))

## GMM

In [ ]:
from scipy.stats import multivariate_normal
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue 
        scores=[]
        clusters=[]
        parameters=[]
        EPSS=[]
        Min_Sampless=[]
        data=(df[[df.columns[i],df.columns[j]]]).to_numpy()
        covariance_type=["full", "tied", "diag", "spherical"]
        for k in  range(0,4):
            gm = GaussianMixture(n_components=2,covariance_type=covariance_type[k]).fit(data)
            x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
            y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
            xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
            Z = np.zeros((xx.shape[0], xx.shape[1]))
            for l in range(gm.n_components):
                density = multivariate_normal(mean=gm.means_[l], cov=gm.covariances_[l]).pdf(np.dstack((xx, yy)))
                Z += density * gm.weights_[l]

            # Plot the resulting contour plot
            plt.subplot(2,2,k+1)
            plt.contour(xx, yy, Z, levels=10, linewidths=1, colors='black')
            plt.contourf(xx, yy, Z, levels=10, cmap='tab20b')
            plt.scatter(data[:, 0], data[:, 1], c=gm.predict(data), cmap='tab20b')
            plt.title('Gaussian Mixture Clustering using %s' % covariance_type[k]+ " for "+Features_string)

## After Normalization

In [ ]:
scaler = preprocessing.StandardScaler()
data=scaler.fit_transform(df)
df = pd.DataFrame(data,
                     columns= iris_data['feature_names'] )
df

### Kmeans

In [ ]:
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(50, 25))
            plt.title(Features_string)
            
        else:
            continue
        inertias = []
        scores = []
        clusters_string=[]
        clusters=[]
        for num_clusters in range(2,18):
            kmeans = KMeans(n_clusters=num_clusters,random_state=42)
            kmeans.fit(df[[df.columns[i],df.columns[j]]])
            inertias.append(kmeans.inertia_)
            scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]], kmeans.labels_, metric='euclidean'))
            clusters_string=str(df.columns[i])+" and "+str(df.columns[j])+" for "+str(num_clusters)+" Clusters "
            clusters.append(len(np.unique(kmeans.labels_)))
            plt.subplot(5, 4, num_clusters-1)
            plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=kmeans.labels_)
            plt.title(clusters_string)
        plt.subplot(5,4,num_clusters)
        plt.plot(range(2,num_clusters+1), inertias, marker='o')
        plt.title('Elbow method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Inertia')
        plt.subplot(5,4,num_clusters+1)
        plt.plot(range(2,num_clusters+1), scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print("Based on silhouette scores, the best number of clusters for "+ Features_string +" is "+str(clusters[scores.index(max(scores))]))

### Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
scores=[]
clusters=[]
parameters=[]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        linkage_array=["average","single"]
        affinity_array=["euclidean","manhattan","cosine"]
        z=1
        for k in range(1,3):
            link=linkage_array[k-1]
            for l in range (1,4):
                a=affinity_array[l-1]
                if link=="average" and a=="cosine":
                    start=0.3
                    stop=0.8
                    step=0.1
                elif link=="single" and a=="cosine":
                    start=0.05
                    stop=0.1
                    step=0.01            
                elif link=="single" and (a=="manhattan" or a=="euclidean"):
                    start=0.05
                    stop=0.1
                    step=0.01
                else:
                    start=1
                    stop=6
                    step=1
                for m in np.arange (start,stop,step):
                    d=m
                    clt = AgglomerativeClustering(linkage=link, affinity=a,distance_threshold=d,n_clusters=None)
                    model = clt.fit(df[[df.columns[i],df.columns[j]]])
                    if (len(np.unique(model.labels_)))>1:
                        plt.subplot(13, 5,z)
                        plot_dendrogram(model, truncate_mode="level", p=8)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        plt.subplot(13, 5,z+1)
                        plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=model.labels_)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                        clusters.append(len(np.unique(model.labels_)))
                        parameters.append(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        z=z+2
        plt.subplot(13,5,z)
        plt.plot(clusters, scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))


### DBSCAN

In [ ]:
scores=[]
clusters=[]
parameters=[]
EPSS=[]
Min_Sampless=[]
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        EPSS=[]
        Min_Sampless=[]
        z=1
        for Min_Samples in range (5,26):
            for EPS in np.arange (0.1,4,0.3):
                model = (DBSCAN(eps=EPS, min_samples=Min_Samples)).fit(df[[df.columns[i],df.columns[j]]])
                if (len(np.unique(model.labels_)))>1:
                    plt.subplot(21, 10,z)
                    plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=model.labels_)
                    plt.title(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                    clusters.append(len(np.unique(model.labels_)))
                    EPSS.append(EPS)
                    Min_Sampless.append(Min_Samples)
                    parameters.append(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    z=z+1
        plt.subplot(21,10,z)
        plt.plot(EPSS, scores, marker='o')
        plt.title("EPS Variation with Silhouette Score ")
        plt.xlabel('EPS')
        plt.ylabel('Score')
        plt.subplot(21,10,z+1)
        plt.plot(Min_Sampless, scores, marker='o')
        plt.title("Min_points Variation with Silhouette Score ")
        plt.xlabel('Min_points')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))

### GMM

In [ ]:
from scipy.stats import multivariate_normal
for i in range(0,4):
    for j in range(0,4):
        if (i!=j):
            Features_string=str(df.columns[i])+" and "+ str(df.columns[j])
            plt.figure(figsize=(100, 50))
        else:
            continue 
        scores=[]
        clusters=[]
        parameters=[]
        EPSS=[]
        Min_Sampless=[]
        data=(df[[df.columns[i],df.columns[j]]]).to_numpy()
        covariance_type=["full", "tied", "diag", "spherical"]
        for k in  range(0,4):
            gm = GaussianMixture(n_components=2,covariance_type=covariance_type[k]).fit(data)
            x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
            y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
            xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
            Z = np.zeros((xx.shape[0], xx.shape[1]))
            for l in range(gm.n_components):
                density = multivariate_normal(mean=gm.means_[l], cov=gm.covariances_[l]).pdf(np.dstack((xx, yy)))
                Z += density * gm.weights_[l]

            # Plot the resulting contour plot
            plt.subplot(2,2,k+1)
            plt.contour(xx, yy, Z, levels=10, linewidths=1, colors='black')
            plt.contourf(xx, yy, Z, levels=10, cmap='tab20b')
            plt.scatter(data[:, 0], data[:, 1], c=gm.predict(data), cmap='tab20b')
            plt.title('Gaussian Mixture Clustering using %s' % covariance_type[k]+ " for "+Features_string)

#### Normalized data shows better results

## Customer dataset
Data preparation and normalization

In [ ]:
df = pd.read_csv('Customer data.csv',index_col='ID')
df.dropna(inplace=True)
#remove the first column as it is unique to each customer and won't be useful for clustering 
#normailize
scaler = preprocessing.StandardScaler()
df[['Sex_n','Martial status_n','Age_n','Education_n','Income_n','Ocupation_n','Sttlement size_n']]=scaler.fit_transform(df[df.columns])

df


### Kmeans

In [ ]:
for i in range(7,14):
    for j in range(7,14):
        if (i!=j):
            Features_string=str(df.columns[i-7])+" and "+str(df.columns[j-7])
            plt.figure(figsize=(50, 25))
            plt.title(Features_string)
            
        else:
            continue
        inertias = []
        scores = []
        clusters_string=[]
        clusters=[]
        for num_clusters in range(2,18):
            kmeans = KMeans(n_clusters=num_clusters,random_state=42)
            kmeans.fit(df[[df.columns[i],df.columns[j]]])
            inertias.append(kmeans.inertia_)
            scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]], kmeans.labels_, metric='euclidean'))
            clusters_string=str(df.columns[i])+" and "+str(df.columns[j])+" for "+str(num_clusters)+" Clusters "
            clusters.append(len(np.unique(kmeans.labels_)))
            plt.subplot(5, 4, num_clusters-1)
            plt.scatter(x=df[df.columns[i]],y=df[df.columns[j]],c=kmeans.labels_)
            plt.title(clusters_string)
        plt.subplot(5,4,num_clusters)
        plt.plot(range(2,num_clusters+1), inertias, marker='o')
        plt.title('Elbow method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Inertia')
        plt.subplot(5,4,num_clusters+1)
        plt.plot(range(2,num_clusters+1), scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print("Based on silhouette scores, the best number of clusters for "+ Features_string +" is "+str(clusters[scores.index(max(scores))]))

##### In order to get  the best number of clusters for each features combination, use the silhouette score (most silhouette score) and elbow methods

### Hierarchal Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
scores=[]
clusters=[]
parameters=[]
for i in range(7,14):
    for j in range(7,14):
        if (i!=j):
            Features_string=str(df.columns[i-7])+" and "+ str(df.columns[j-7])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        linkage_array=["average","single"]
        affinity_array=["euclidean","manhattan","cosine"]
        z=1
        for k in range(1,3):
            link=linkage_array[k-1]
            for l in range (1,4):
                a=affinity_array[l-1]
                if link=="average" and a=="cosine":
                    start=0.3
                    stop=0.8
                    step=0.1
                elif link=="single" and a=="cosine":
                    start=0.05
                    stop=0.1
                    step=0.01            
                elif link=="single" and (a=="manhattan" or a=="euclidean"):
                    start=0.05
                    stop=0.1
                    step=0.01
                else:
                    start=1
                    stop=6
                    step=1
                for m in np.arange (start,stop,step):
                    d=m
                    clt = AgglomerativeClustering(linkage=link, affinity=a,distance_threshold=d,n_clusters=None)
                    model = clt.fit(df[[df.columns[i],df.columns[j]]])
                    if (len(np.unique(model.labels_)))>1:
                        plt.subplot(13, 5,z)
                        plot_dendrogram(model, truncate_mode="level", p=8)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        plt.subplot(13, 5,z+1)
                        plt.scatter(x=df[df.columns[i-7]],y=df[df.columns[j-7]],c=model.labels_)
                        plt.title(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                        clusters.append(len(np.unique(model.labels_)))
                        parameters.append(str(link)+" , "+str(a)+" , "+str(d)+" for "+ Features_string)
                        z=z+2
        plt.subplot(13,5,z)
        plt.plot(clusters, scores, marker='o')
        plt.title('Silhouette method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))


### DBSCAN

In [ ]:
scores=[]
clusters=[]
parameters=[]
EPSS=[]
Min_Sampless=[]
for i in range(7,14):
    for j in range(7,14):
        if (i!=j):
            Features_string=str(df.columns[i-7])+" and "+ str(df.columns[j-7])
            plt.figure(figsize=(100, 50))
        else:
            continue
        scores=[]
        clusters=[]
        parameters=[]
        EPSS=[]
        Min_Sampless=[]
        z=1
        for Min_Samples in range (5,26):
            for EPS in np.arange (0.1,4,0.3):
                model = (DBSCAN(eps=EPS, min_samples=Min_Samples)).fit(df[[df.columns[i],df.columns[j]]])
                if (len(np.unique(model.labels_)))>1:
                    plt.subplot(21, 10,z)
                    plt.scatter(x=df[df.columns[i-7]],y=df[df.columns[j-7]],c=model.labels_)
                    plt.title(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    scores.append(silhouette_score(df[[df.columns[i],df.columns[j]]],model.labels_ ))
                    clusters.append(len(np.unique(model.labels_)))
                    EPSS.append(EPS)
                    Min_Sampless.append(Min_Samples)
                    parameters.append(str(EPS)+" , "+str(Min_Samples)+" for "+ Features_string)
                    z=z+1
        plt.subplot(21,10,z)
        plt.plot(EPSS, scores, marker='o')
        plt.title("EPS Variation with Silhouette Score ")
        plt.xlabel('EPS')
        plt.ylabel('Score')
        plt.subplot(21,10,z+1)
        plt.plot(Min_Sampless, scores, marker='o')
        plt.title("Min_points Variation with Silhouette Score ")
        plt.xlabel('Min_points')
        plt.ylabel('Score')
        print( "number of clusters= " + str(clusters[scores.index(max(scores))])+
              " and parameters "+str(parameters[scores.index(max(scores))]))

### GMM

In [ ]:
#For income and Age only 
from scipy.stats import multivariate_normal
scores=[]
clusters=[]
parameters=[]
EPSS=[]
Min_Sampless=[]

Features_string=str(df.columns[9-7])+" and "+ str(df.columns[11-7])
data=(df[[df.columns[9],df.columns[11]]]).to_numpy()
plt.figure(figsize=(100, 50))
covariance_type=["full", "tied", "diag", "spherical"]
for k in  range(0,4):
    gm = GaussianMixture(n_components=2,covariance_type=covariance_type[k]).fit(data)
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    Z = np.zeros((xx.shape[0], xx.shape[1]))
    for l in range(gm.n_components):
        density = multivariate_normal(mean=gm.means_[l], cov=gm.covariances_[l]).pdf(np.dstack((xx, yy)))
        Z += density * gm.weights_[l]

    # Plot the resulting contour plot
    plt.subplot(2,2,k+1)
    plt.contour(xx, yy, Z, levels=10, linewidths=1, colors='black')
    plt.contourf(xx, yy, Z, levels=10, cmap='tab20b')
    plt.scatter(data[:, 0], data[:, 1], c=gm.predict(data), cmap='tab20b')
    plt.title('Gaussian Mixture Clustering using %s' % covariance_type[k]+ " for "+Features_string)